In [16]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller

filename = 'agent_log_0'
df = pd.read_csv("../sosi/outputs/" + filename + ".csv")

types = range(len(df['type'].unique()))
num_ticks = int(df["tick"].max()) + 1

# Group by type and tick, and count individuals
count_df = df.groupby(['type', 'tick']).size().reset_index(name='count')

# Function to perform ADF test on count data within a window of 20 ticks
def mean_adf_p_value_within_window(start_tick, type_data):
    window_ticks = range(start_tick, start_tick + 20)
    mean_p_value = 0.0
    if all(tick in type_data['tick'].values for tick in window_ticks):
        window_data = list(type_data[type_data['tick'].isin(window_ticks)]['count'])
        if(len(set(window_data))==1):
            window_data[0]+=1
        if(len(window_data) != 20):
            print(len(window_data))
        result = adfuller(window_data)
        mean_p_value = result[1]
    return mean_p_value

# Calculate mean ADF p-values for each sliding window of 20 ticks for each type
min_mean_p_value_global = float('inf')
best_window_start_tick_global = None
best_type_global = None
for t in types:
    type_data = count_df[count_df['type'] == t]
    min_mean_p_value = float('inf')
    best_window_start_tick = None
    for start_tick in range(num_ticks - 19):  # Ensure the window stays within bounds
        avg_p_value = mean_adf_p_value_within_window(start_tick, type_data)
        if avg_p_value < min_mean_p_value:
            min_mean_p_value = avg_p_value
            best_window_start_tick = start_tick
    if min_mean_p_value < min_mean_p_value_global:
        min_mean_p_value_global = min_mean_p_value
        best_window_start_tick_global = best_window_start_tick
        best_type_global = t

# Print the global best tick
print(f"Global best tick: {best_window_start_tick_global} for type {best_type_global} with mean ADF p-value: {min_mean_p_value_global}")

/home/timovdk/soil_sim/.venv/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:958: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/home/timovdk/soil_sim/.venv/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:958: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/home/timovdk/soil_sim/.venv/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:958: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/home/timovdk/soil_sim/.venv/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:958: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/home/timovdk/soil_sim/.venv/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:958: RuntimeWarning: divide by 

Global best tick: 12 for type 0 with mean ADF p-value: 0.0
